In [ ]:
from utils import *

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import pandas as pd

from ortools.sat.python import cp_model
from ortools.sat.python.cp_model import _SumArray

In [ ]:
def solve(raw_score, full_score):
    model = cp_model.CpModel()
    eq_dict, sym_dict = addAllVars(turn)

    buildScoreConstraint(model, eq_dict, full_score, True)
    buildScoreConstraint(model, eq_dict, raw_score, False)
    model.Add(_SumArray([sym_dict[e] for e in values['units'].keys()]) < 20)


    solver = cp_model.CpSolver()
    solution_printer = DataFrameSolutionAggregator(sym_dict.values())
    status = solver.SearchForAllSolutions(model, solution_printer)
    return solution_printer

In [1]:
# superb_ruin
combined_scores = [
    ([515, 515], [390, 390]),
    ([565, 565], [440, 440]),
    ([590, 600], [450, 450]),
    ([855, 635], [700, 460]),
    ([1385, 670], [1230, 460]),
    ([1860, 1215], [1700, 990]),
    ([2230, 1815], [2010, 1530])
]

In [2]:
from score_utils import *
possible_score = getPossibleScores(combined_scores)
len(possible_score)

2

In [3]:
possible_score

[[[[515, 390],
   [565, 440],
   [590, 450],
   [855, 700],
   [1385, 1230],
   [1860, 1700],
   [2230, 2010]],
  [[515, 390],
   [565, 440],
   [600, 450],
   [635, 460],
   [670, 460],
   [1215, 990],
   [1815, 1530]]],
 [[[515, 390],
   [565, 440],
   [590, 450],
   [635, 460],
   [670, 460],
   [1215, 990],
   [1815, 1530]],
  [[515, 390],
   [565, 440],
   [600, 450],
   [855, 700],
   [1385, 1230],
   [1860, 1700],
   [2230, 2010]]]]

In [7]:
player_scores = [
    [565, 440],
    [590, 450],
    [855, 700],
    [1385, 1230],
    [1860, 1700],
    [2230, 2010]
]

In [8]:
player_2_scores = [
    [565, 440],
    [600, 450],
    [635, 460],
    [670, 460],
    [1215, 990],
    [1815, 1530]
]

# Equations: 

full score = score(units) + score(tech) + score(city) + score(vision)

raw score =  score(units) + score(tech) + score(city)

'remaining stars + spt + extra stars = stars(units) + stars(tech) + star(building)'

for v in list(dic.values()):
    val = solver.Value(v)
    if val != 0:
        print(f'{v}: {val}')

In [1]:
import re
from ortools.sat.python import cp_model
from SolutionAggregator import SolutionAggregator
from utils import *
from file_utils import *
from constr_utils import buildAllConstraints

import pandas as pd
pd.set_option('display.max_columns', None)

scores = {
    "start": [515, 390],
    "t0": [565, 440],
    "t1": [600, 450],
    "t2": [635, 460],
    "t3": [670, 460],
    "t4": [1215, 990],
    "t5": [1815, 1530]
}
turn_list = ["start", "t0", "t1", "t2", "t3"]#, "t4", "t5"]
for turn in turn_list:
    buildDictionnaries(turn)
    full_score = scores[turn][0] 
    raw_score = scores[turn][1]

    buildAllConstraints(turn)

    setRawScore(turn, raw_score)
    setFullScore(turn, full_score)
    setTribe(turn, "Kickoo")

    addInitialState(turn)

# addSolution(turn)
# model.Minimize(
#     sum(
#         get(turn, "ruin") +
#         sum(get(turn, "killed_" + u) for u in getValues()['units'] if u != 'giant') 
#         for turn in turn_list))

solver = cp_model.CpSolver()
output_variables = list(dic.values())
solution_printer = SolutionAggregator(output_variables)
#status = solver.Solve(model)
status = solver.SearchForAllSolutions(model, solution_printer)
print('Solve status: %s' % solver.StatusName(status))

Solve status: OPTIMAL


In [3]:
x = solution_printer.get_data_frame()
x.shape

(1473, 157)

In [ ]:
dropColumns(x)

In [4]:
printValueList(x)

start_warrior 1 [1]
start_level-1 1 [1]
start_tier-1 1 [1]
start_claimed 1 [9]
start_revealed 1 [25]
start_hunting 1 [1]
start_Bardur 1 [1]
start_raw_score 1 [390]
start_full_score 1 [515]
start_level-1_count 1 [1]
start_pop_max 1 [1]
start_stars 1 [3]
start_spt 1 [2]
start_star_spending 1 [5]
start_tier-1_cost 1 [5]
start_tier-1_stars 1 [5]
start_tier-2_stars 1 [6]
start_tier-3_stars 1 [7]
start_city_spt 1 [2]
start_tech_stars 1 [5]
t0_warrior 1 [1]
t0_level-1 1 [1]
t0_level-2 1 [1]
t0_population 1 [2]
t0_tier-1 1 [1]
t0_claimed 1 [9]
t0_revealed 1 [25]
t0_hunting 1 [1]
t0_Bardur 1 [1]
t0_raw_score 1 [440]
t0_full_score 1 [565]
t0_claimed_cst 1 [1]
t0_lvl_1_cst 1 [1]
t0_ruin 2 [1 0]
t0_ruin_giant 2 [1 0]
t0_ruin_stars 2 [0 1]
t0_level-2_spt 1 [1]
t0_level-2_count 1 [1]
t0_pop_min 1 [2]
t0_pop_max 1 [4]
t0_animal 1 [2]
t0_stars 2 [ 1 11]
t0_spt 1 [4]
t0_star_income 2 [ 2 12]
t0_star_spending 1 [4]
t0_animal_stars 1 [4]
t0_tier-1_cost 1 [5]
t0_tier-1_stars 1 [5]
t0_tier-2_stars 1 [6]
t0

2770135983297921024

In [5]:
cleaned = chooseBestSolution(x)
cleaned.shape

(9, 157)

In [6]:
cleaned

,start_warrior,start_level-1,start_tier-1,start_claimed,start_revealed,start_hunting,start_Bardur,start_raw_score,start_full_score,start_level-1_count,start_pop_max,start_stars,start_spt,start_star_spending,start_tier-1_cost,start_tier-1_stars,start_tier-2_stars,start_tier-3_stars,start_city_spt,start_tech_stars,t0_warrior,t0_level-1,t0_level-2,t0_population,t0_tier-1,t0_claimed,t0_revealed,t0_hunting,t0_Bardur,t0_raw_score,t0_full_score,t0_claimed_cst,t0_lvl_1_cst,t0_ruin,t0_ruin_giant,t0_ruin_stars,t0_level-2_spt,t0_level-2_count,t0_pop_min,t0_pop_max,t0_animal,t0_stars,t0_spt,t0_star_income,t0_star_spending,t0_animal_stars,t0_tier-1_cost,t0_tier-1_stars,t0_tier-2_stars,t0_tier-3_stars,t0_city_spt,t0_pop_stars,t1_warrior,t1_giant,t1_level-1,t1_level-2,t1_population,t1_tier-1,t1_claimed,t1_revealed,t1_hunting,t1_Bardur,t1_raw_score,t1_full_score,t1_claimed_cst,t1_lvl_1_cst,t1_ruin,t1_ruin_giant,t1_ruin_stars,t1_level-2_spt,t1_level-2_count,t1_pop_min,t1_pop_max,t1_animal,t1_stars,t1_spt,t1_star_income,t1_star_spending,t1_animal_stars,t1_tier-1_cost,t1_tier-1_stars,t1_tier-2_stars,t1_tier-3_stars,t1_city_spt,t1_pop_stars,t2_warrior,t2_giant,t2_level-1,t2_level-2,t2_population,t2_tier-1,t2_claimed,t2_revealed,t2_trained_warrior,t2_hunting,t2_Bardur,t2_raw_score,t2_full_score,t2_claimed_cst,t2_lvl_1_cst,t2_ruin,t2_ruin_giant,t2_ruin_stars,t2_level-2_spt,t2_level-2_count,t2_pop_min,t2_pop_max,t2_animal,t2_stars,t2_spt,t2_star_income,t2_star_spending,t2_animal_stars,t2_warrior_stars,t2_tier-1_cost,t2_tier-1_stars,t2_tier-2_stars,t2_tier-3_stars,t2_city_spt,t2_unit_stars,t2_pop_stars,t3_warrior,t3_giant,t3_level-1,t3_level-2,t3_population,t3_tier-1,t3_claimed,t3_revealed,t3_trained_warrior,t3_hunting,t3_Bardur,t3_raw_score,t3_full_score,t3_claimed_cst,t3_lvl_1_cst,t3_ruin,t3_ruin_giant,t3_ruin_stars,t3_level-2_spt,t3_level-2_count,t3_pop_min,t3_pop_max,t3_animal,t3_stars,t3_spt,t3_star_income,t3_star_spending,t3_animal_stars,t3_warrior_stars,t3_tier-1_cost,t3_tier-1_stars,t3_tier-2_stars,t3_tier-3_stars,t3_city_spt,t3_unit_stars,t3_pop_stars
124,1,1,1,9,25,1,1,390,515,1,1,3,2,5,5,5,6,7,2,5,1,1,1,2,1,9,25,1,1,440,565,1,1,0,0,0,1,1,2,4,2,1,4,2,4,4,5,5,6,7,3,4,1,0,1,1,4,1,9,30,1,1,450,600,1,1,0,0,0,1,1,2,4,4,1,4,4,4,8,5,5,6,7,3,4,1,1,1,1,4,1,9,35,0,1,1,460,635,1,1,0,0,0,1,1,2,4,4,5,4,4,0,8,0,5,5,6,7,3,0,0,1,1,1,1,4,1,9,42,0,1,1,460,670,1,1,0,0,0,1,1,2,4,4,9,4,4,0,8,0,5,5,6,7,3,0,0
127,1,1,1,9,25,1,1,390,515,1,1,3,2,5,5,5,6,7,2,5,1,1,1,2,1,9,25,1,1,440,565,1,1,0,0,0,1,1,2,4,2,1,4,2,4,4,5,5,6,7,3,4,1,0,1,1,4,1,9,30,1,1,450,600,1,1,0,0,0,1,1,2,4,4,1,4,4,4,8,5,5,6,7,3,4,1,1,1,1,4,1,9,35,0,1,1,460,635,1,1,0,0,0,1,1,2,4,4,5,4,4,0,8,0,5,5,6,7,3,0,0,2,0,1,1,4,1,9,42,1,1,1,460,670,1,1,0,0,0,1,1,2,4,4,7,4,4,2,8,2,5,5,6,7,3,2,0
136,1,1,1,9,25,1,1,390,515,1,1,3,2,5,5,5,6,7,2,5,1,1,1,2,1,9,25,1,1,440,565,1,1,0,0,0,1,1,2,4,2,1,4,2,4,4,5,5,6,7,3,4,1,1,1,1,2,1,9,30,1,1,450,600,1,1,0,0,0,1,1,2,4,2,5,4,4,0,4,5,5,6,7,3,0,1,2,1,1,2,1,9,35,0,1,1,460,635,1,1,0,0,0,1,1,2,4,2,9,4,4,0,4,0,5,5,6,7,3,0,0,1,2,1,1,2,1,9,42,0,1,1,460,670,1,1,0,0,0,1,1,2,4,2,13,4,4,0,4,0,5,5,6,7,3,0,0
137,1,1,1,9,25,1,1,390,515,1,1,3,2,5,5,5,6,7,2,5,1,1,1,2,1,9,25,1,1,440,565,1,1,0,0,0,1,1,2,4,2,1,4,2,4,4,5,5,6,7,3,4,1,1,1,1,2,1,9,30,1,1,450,600,1,1,0,0,0,1,1,2,4,2,5,4,4,0,4,5,5,6,7,3,0,1,2,1,1,2,1,9,35,0,1,1,460,635,1,1,0,0,0,1,1,2,4,2,9,4,4,0,4,0,5,5,6,7,3,0,0,1,1,1,1,4,1,9,42,0,1,1,460,670,1,1,0,0,0,1,1,2,4,4,9,4,4,4,8,0,5,5,6,7,3,0,4
138,1,1,1,9,25,1,1,390,515,1,1,3,2,5,5,5,6,7,2,5,1,1,1,2,1,9,25,1,1,440,565,1,1,0,0,0,1,1,2,4,2,1,4,2,4,4,5,5,6,7,3,4,1,1,1,1,2,1,9,30,1,1,450,600,1,1,0,0,0,1,1,2,4,2,5,4,4,0,4,5,5,6,7,3,0,1,1,1,1,4,1,9,35,0,1,1,460,635,1,1,0,0,0,1,1,2,4,4,5,4,4,4,8,0,5,5,6,7,3,0,4,1,1,1,1,4,1,9,42,0,1,1,460,670,1,1,0,0,0,1,1,2,4,4,9,4,4,0,8,0,5,5,6,7,3,0,0
139,1,1,1,9,25,1,1,390,515,1,1,3,2,5,5,5,6,7,2,5,1,1,1,2,1,9,25,1,1,440,565,1,1,0,0,0,1,1,2,4,2,1,4,2,4,4,5,5,6,7,3,4,1,1,1,1,2,1,9,30,1,1,450,600,1,1,0,0,0,1,1,2,4,2,5,4,4,0,4,5,5,6,7,3,0,2,1,1,1,2,1,9,35,1,1,1

In [7]:
dropColumns(cleaned.drop_duplicates())

,t1_giant,t1_population,t1_animal,t1_stars,t1_star_spending,t1_animal_stars,t1_pop_stars,t2_warrior,t2_giant,t2_population,t2_trained_warrior,t2_animal,t2_stars,t2_star_spending,t2_animal_stars,t2_warrior_stars,t2_unit_stars,t2_pop_stars,t3_warrior,t3_giant,t3_population,t3_trained_warrior,t3_animal,t3_stars,t3_star_spending,t3_animal_stars,t3_warrior_stars,t3_unit_stars,t3_pop_stars
124,0,4,4,1,4,8,4,1,1,4,0,4,5,0,8,0,0,0,1,1,4,0,4,9,0,8,0,0,0
127,0,4,4,1,4,8,4,1,1,4,0,4,5,0,8,0,0,0,2,0,4,1,4,7,2,8,2,2,0
136,1,2,2,5,0,4,0,1,2,2,0,2,9,0,4,0,0,0,1,2,2,0,2,13,0,4,0,0,0
137,1,2,2,5,0,4,0,1,2,2,0,2,9,0,4,0,0,0,1,1,4,0,4,9,4,8,0,0,4
138,1,2,2,5,0,4,0,1,1,4,0,4,5,4,8,0,0,4,1,1,4,0,4,9,0,8,0,0,0
139,1,2,2,5,0,4,0,2,1,2,1,2,7,2,4,2,2,0,3,0,2,1,2,11,0,4,2,0,0
140,1,2,2,5,0,4,0,1,2,2,0,2,9,0,4,0,0,0,2,1,2,1,2,11,2,4,2,2,0
141,1,2,2,5,0,4,0,1,2,2,0,2,9,0,4,0,0,0,2,0,4,1,4,7,6,8,2,2,4
142,1,2,2,5,0,4,0,1,1,4,0,4,5,4,8,0,0,4,2,0,4,1,4,7,2,8,2,2,0


In [ ]:
printScenario(x)

In [8]:
variants = printVariants(x)

t0_warrior 2 [1 0]
t0_giant 2 [0 1]
t0_trained_warrior 2 [0 1]
t0_killed_warrior 2 [0 1]
t0_ruin 2 [0 1]
t0_ruin_giant 2 [0 1]
t0_ruin_stars 2 [0 1]
t0_stars 3 [ 1 11  9]
t0_star_income 2 [ 2 12]
t0_star_spending 2 [4 6]
t0_warrior_stars 2 [0 2]
t0_unit_stars 2 [0 2]
t1_warrior 3 [1 0 2]
t1_giant 3 [0 1 2]
t1_population 2 [4 2]
t1_trained_warrior 2 [0 1]
t1_killed_warrior 2 [0 1]
t1_ruin 2 [0 1]
t1_ruin_giant 2 [0 1]
t1_ruin_stars 2 [0 1]
t1_animal 2 [4 2]
t1_stars 7 [ 1  5  3 15 11 13  9]
t1_star_income 2 [ 4 14]
t1_star_spending 4 [4 0 2 6]
t1_animal_stars 2 [8 4]
t1_warrior_stars 2 [0 2]
t1_unit_stars 2 [0 2]
t1_pop_stars 2 [4 0]
t2_warrior 4 [0 1 2 3]
t2_giant 4 [2 3 1 0]
t2_population 2 [4 2]
t2_trained_warrior 2 [0 1]
t2_killed_warrior 3 [1 0 2]
t2_ruin 3 [0 1 2]
t2_ruin_giant 3 [0 1 2]
t2_ruin_stars 3 [0 1 2]
t2_animal 2 [4 2]
t2_stars 12 [ 5  9  7 19 29 15 25  3 17 13 27 23]
t2_star_income 3 [ 4 14 24]
t2_star_spending 4 [0 4 2 6]
t2_animal_stars 2 [8 4]
t2_warrior_stars 2 [0 2

In [ ]:
variants

In [9]:
printVariants(cleaned)

t0_warrior 2 [1 0]
t0_giant 2 [0 1]
t0_killed_warrior 2 [0 1]
t1_warrior 3 [1 0 2]
t1_giant 3 [0 1 2]
t1_population 2 [4 2]
t1_trained_warrior 2 [0 1]
t1_killed_warrior 2 [0 1]
t1_animal 2 [4 2]
t1_stars 3 [1 5 3]
t1_star_spending 3 [4 0 2]
t1_animal_stars 2 [8 4]
t1_warrior_stars 2 [0 2]
t1_unit_stars 2 [0 2]
t1_pop_stars 2 [4 0]
t2_warrior 4 [0 1 2 3]
t2_giant 4 [2 3 1 0]
t2_population 2 [4 2]
t2_trained_warrior 2 [0 1]
t2_killed_warrior 3 [1 0 2]
t2_animal 2 [4 2]
t2_stars 4 [5 9 7 3]
t2_star_spending 4 [0 4 2 6]
t2_animal_stars 2 [8 4]
t2_warrior_stars 2 [0 2]
t2_unit_stars 2 [0 2]
t2_pop_stars 2 [0 4]
t3_warrior 4 [0 1 2 3]
t3_giant 4 [2 1 3 0]
t3_population 2 [4 2]
t3_trained_warrior 2 [0 1]
t3_killed_warrior 4 [0 1 2 3]
t3_animal 2 [4 2]
t3_stars 4 [ 9  7 13 11]
t3_star_spending 4 [0 2 4 6]
t3_animal_stars 2 [8 4]
t3_warrior_stars 2 [0 2]
t3_unit_stars 2 [0 2]
t3_pop_stars 2 [0 4]


[('t0_warrior', array([1, 0])),
 ('t0_giant', array([0, 1])),
 ('t0_killed_warrior', array([0, 1])),
 ('t1_warrior', array([1, 0, 2])),
 ('t1_giant', array([0, 1, 2])),
 ('t1_population', array([4, 2])),
 ('t1_trained_warrior', array([0, 1])),
 ('t1_killed_warrior', array([0, 1])),
 ('t1_animal', array([4, 2])),
 ('t1_stars', array([1, 5, 3])),
 ('t1_star_spending', array([4, 0, 2])),
 ('t1_animal_stars', array([8, 4])),
 ('t1_warrior_stars', array([0, 2])),
 ('t1_unit_stars', array([0, 2])),
 ('t1_pop_stars', array([4, 0])),
 ('t2_warrior', array([0, 1, 2, 3])),
 ('t2_giant', array([2, 3, 1, 0])),
 ('t2_population', array([4, 2])),
 ('t2_trained_warrior', array([0, 1])),
 ('t2_killed_warrior', array([1, 0, 2])),
 ('t2_animal', array([4, 2])),
 ('t2_stars', array([5, 9, 7, 3])),
 ('t2_star_spending', array([0, 4, 2, 6])),
 ('t2_animal_stars', array([8, 4])),
 ('t2_warrior_stars', array([0, 2])),
 ('t2_unit_stars', array([0, 2])),
 ('t2_pop_stars', array([0, 4])),
 ('t3_warrior', array(

In [ ]:
# def prettyPrint(variants):
#     turns = list(set(map(lambda u: u[0].split("_")[0], variants)))
#     for turn_i in turns:
#         turn_string = ("Turn " + turn_i[1:] + ":" if turn_i.startswith("t") else "Start:")
#         print(turn_string)
#         for element, value in map(lambda u: (" ".join(u[0].split("_")[1:]), u[1]), filter(lambda u: u[0].startswith(turn_i), # variants)):
#             print(element + ":", ", ".join(map(lambda u: str(u), value)))
# prettyPrint(variants)

In [9]:
printValueList(cleaned)

start_warrior 1 [1]
start_level-1 1 [1]
start_tier-1 1 [1]
start_claimed 1 [9]
start_revealed 1 [25]
start_hunting 1 [1]
start_Bardur 1 [1]
start_raw_score 1 [390]
start_full_score 1 [515]
start_level-1_count 1 [1]
start_pop_max 1 [1]
start_stars 1 [3]
start_spt 1 [2]
start_star_spending 1 [5]
start_tier-1_cost 1 [5]
start_tier-1_stars 1 [5]
start_tier-2_stars 1 [6]
start_tier-3_stars 1 [7]
start_city_spt 1 [2]
start_tech_stars 1 [5]
t0_warrior 1 [1]
t0_level-1 1 [1]
t0_level-2 1 [1]
t0_population 1 [2]
t0_tier-1 1 [1]
t0_claimed 1 [9]
t0_revealed 1 [25]
t0_hunting 1 [1]
t0_Bardur 1 [1]
t0_raw_score 1 [440]
t0_full_score 1 [565]
t0_claimed_cst 1 [1]
t0_lvl_1_cst 1 [1]
t0_ruin 1 [0]
t0_ruin_giant 1 [0]
t0_ruin_stars 1 [0]
t0_level-2_spt 1 [1]
t0_level-2_count 1 [1]
t0_pop_min 1 [2]
t0_pop_max 1 [4]
t0_animal 1 [2]
t0_stars 1 [1]
t0_spt 1 [4]
t0_star_income 1 [2]
t0_star_spending 1 [4]
t0_animal_stars 1 [4]
t0_tier-1_cost 1 [5]
t0_tier-1_stars 1 [5]
t0_tier-2_stars 1 [6]
t0_tier-3_stars 

77309411328

In [ ]:
printValueList(x)

In [ ]:
{
		"name": "water_temple",
		"cost": 20,
		"pop": 1,
		"requires": "acquatism"
	},{
		"name": "forest_temple",
		"cost": 20,
		"pop": 1,
		"requires": "spiritualism"
	},{
		"name": "mountain_temple",
		"cost": 20,
		"pop": 1,
		"requires": "meditation"
	},{
		"name": "temple",
		"cost": 20,
		"pop": 1,
		"requires": "free spirit"
	},{
		"name": "road_conection",
		"cost": 0,
		"pop": 2,
		"requires": "roads"
	},{
		"name": "water_connection",
		"cost": 0,
		"pop": 2,
		"requires": "sailing"
	}